### Notebook Set-Up

#### Import Required Modules

In [ ]:
# %pip install sqlalchemy duckdb duckdb-engine cloud-sql-python-connector[pg8000]

In [74]:
import os
import sys
import csv
import json
import duckdb

from typing import List, Dict
from dotenv import load_dotenv
from datetime import datetime

import numpy as np
import pandas as pd

# from pyspark.sql import SparkSession
# from pyspark.sql import functions as f
# from pyspark.sql.window import Window

#### Load Secrets as Environment Variables

In [75]:
load_dotenv()

True

#### Set Workflow Constants

In [76]:
RAW_PATH = "dbfs:/FileStore/data/raw"
CLN_PATH = "dbfs:/FileStore/data/clean"
JSON_PATH = "../data/json"

### Export the Data as CSV Files to Bulk Insert into the Database

#### Import Cleaned Static Data

In [77]:
movies = spark.read.parquet(os.path.join(CLN_PATH, "movies"))
movies.show(5)
movies.count()

NameError: name 'spark' is not defined

In [ ]:
ratings = spark.read.parquet(os.path.join(CLN_PATH, "ratings"))
ratings.show(5)
ratings.count()

+-------+-------+------+-------------------+
|user_id|tmdb_id|rating|          timestamp|
+-------+-------+------+-------------------+
|  42017|     14|   4.0|2015-09-10 22:15:39|
|  54926|    165|   2.5|2006-03-19 20:54:43|
|  47629|   1637|   4.0|1996-07-01 14:27:03|
|  68508|  13920|   5.0|2003-11-14 01:18:56|
|  59910|  24746|   4.0|2017-07-29 19:41:07|
+-------+-------+------+-------------------+
only showing top 5 rows

Out[53]: 26024289

In [ ]:
model_frame = spark.read.parquet(os.path.join(CLN_PATH, "model_frame"))
model_frame.show(5)
model_frame.count()

+-------+-------+------+-------------------+---------+----------+
|user_id|tmdb_id|rating|          timestamp|cnt_users|cnt_movies|
+-------+-------+------+-------------------+---------+----------+
| 171679|   2255|   2.0|2013-04-23 14:39:05|    13641|       133|
| 217667|  10427|   2.0|1996-11-08 17:21:30|     3441|        58|
|  31793|    532|   3.0|1997-05-07 15:14:38|    12698|       107|
|  96489|    197|   3.0|2015-10-25 19:42:18|    61076|        84|
| 198045|    641|   2.5|2015-11-04 04:07:02|    19478|       689|
+-------+-------+------+-------------------+---------+----------+
only showing top 5 rows

Out[54]: 15372729

#### Select the Subset of Users/Movies/Ratings Making it Into the Model Frame

In [ ]:
model_users = model_frame.select('user_id').distinct().withColumn('user_id', f.col('user_id').cast('STRING')).filter(f.col('user_id').isNotNull())
model_users.count()

Out[55]: 195899

In [ ]:
model_movies = model_frame.select('tmdb_id').distinct().withColumn('tmdb_id', f.col('tmdb_id').cast('STRING')).filter(f.col('tmdb_id').isNotNull())
model_movies.count()

Out[56]: 6235

In [ ]:
model_ratings = model_frame \
    .select('user_id', 'tmdb_id').distinct() \
    .withColumn('user_id', f.col('user_id').cast('STRING')) \
    .withColumn('tmdb_id', f.col('tmdb_id').cast('STRING')) \
    .filter(f.col('user_id').isNotNull()) \
    .filter(f.col('tmdb_id').isNotNull())

model_ratings.count() 

Out[57]: 15372666

#### Create Users/Movies/Ratings Datasets for Export

In [ ]:
updated_at = datetime.now()
updated_at

Out[58]: datetime.datetime(2023, 10, 23, 18, 44, 55, 294176)

In [ ]:
export_users = model_users \
    .withColumn('fname', f.lit("ANONYMOUS")) \
    .withColumn('lname', f.lit("ANONYMOUS")) \
    .withColumn('email', f.lit("ANONYMOUS@ANONYMOUS.COM")) \
    .withColumn('updated_at', f.lit(updated_at))

export_users.agg(f.countDistinct('user_id').alias('users'), f.count('*').alias('records')).show()
export_users.printSchema()
export_users.show(5)

+------+-------+
| users|records|
+------+-------+
|195899| 195899|
+------+-------+

root
 |-- user_id: string (nullable = true)
 |-- fname: string (nullable = false)
 |-- lname: string (nullable = false)
 |-- email: string (nullable = false)
 |-- updated_at: timestamp (nullable = false)

+-------+---------+---------+--------------------+--------------------+
|user_id|    fname|    lname|               email|          updated_at|
+-------+---------+---------+--------------------+--------------------+
| 231350|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
| 228408|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
| 209326|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
| 207398|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
|  83250|ANONYMOUS|ANONYMOUS|ANONYMOUS@ANONYMO...|2023-10-23 18:44:...|
+-------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
export_movies = movies \
    .join(model_movies, on='tmdb_id', how='inner') \
    .select("tmdb_id", "title", "release_date", "runtime", "genres", "keywords", "overview", "budget", "revenue", "popularity", "vote_average", "vote_count") \
    .withColumn('updated_at', f.lit(updated_at))

export_movies.agg(f.countDistinct('tmdb_id').alias('movies'), f.count('*').alias('records')).show()
export_movies.printSchema()   
export_movies.show(5)

+------+-------+
|movies|records|
+------+-------+
|  6235|   6235|
+------+-------+

root
 |-- tmdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- runtime: double (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- overview: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- revenue: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- updated_at: timestamp (nullable = false)

+-------+--------------------+------------+-------+--------------------+--------------------+--------------------+---------+---------+----------+------------+----------+--------------------+
|tmdb_id|               title|release_date|runtime|              genres|            keywords| 

In [ ]:
export_ratings = ratings \
    .join(model_ratings, on=["user_id", "tmdb_id"], how='inner') \
    .withColumn("rank", f.row_number().over(Window.partitionBy("user_id", "tmdb_id").orderBy(f.rand(seed=1492)))) \
    .filter(f.col("rank") == 1) \
    .select("user_id", "tmdb_id", "rating") \
    .withColumn('updated_at', f.lit(updated_at))

export_ratings.agg(f.countDistinct('user_id', 'tmdb_id').alias('ratings'), f.count('*').alias('records')).show()
export_ratings.printSchema()   
export_ratings.show(5)

+--------+--------+
| ratings| records|
+--------+--------+
|15372666|15372666|
+--------+--------+

root
 |-- user_id: string (nullable = true)
 |-- tmdb_id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- updated_at: timestamp (nullable = false)

+-------+-------+------+--------------------+
|user_id|tmdb_id|rating|          updated_at|
+-------+-------+------+--------------------+
| 100000|    280|   4.0|2023-10-23 18:44:...|
| 100001|  10158|   4.0|2023-10-23 18:44:...|
| 100002|  10923|   3.5|2023-10-23 18:44:...|
| 100002|  11529|   4.0|2023-10-23 18:44:...|
| 100002|    120|   4.0|2023-10-23 18:44:...|
+-------+-------+------+--------------------+
only showing top 5 rows



#### Export the Static User/Movie/Rating Data to JSONLINES

In [ ]:
export_users.toPandas().to_json(os.path.join(JSON_PATH, "users.json"), orient="records", lines=True, date_format="iso")

In [ ]:
export_movies.toPandas().to_json(os.path.join(JSON_PATH, "movies.json"), orient="records", lines=True, date_format="iso")

In [ ]:
export_ratings.toPandas().to_json(os.path.join(JSON_PATH, "ratings.json"), orient="records", lines=True, date_format="iso")

### Load Static Data into the Application Database

#### Import Static Data in JSONLines Format

In [78]:
users = pd.read_json(os.path.join(JSON_PATH, "users.json"), orient="records", lines=True)
users.head()

,user_id,fname,lname,email,updated_at
0,231350,ANONYMOUS,ANONYMOUS,ANONYMOUS@ANONYMOUS.COM,2023-10-23 18:44:55.294000+00:00
1,228408,ANONYMOUS,ANONYMOUS,ANONYMOUS@ANONYMOUS.COM,2023-10-23 18:44:55.294000+00:00
2,209326,ANONYMOUS,ANONYMOUS,ANONYMOUS@ANONYMOUS.COM,2023-10-23 18:44:55.294000+00:00
3,207398,ANONYMOUS,ANONYMOUS,ANONYMOUS@ANONYMOUS.COM,2023-10-23 18:44:55.294000+00:00
4,83250,ANONYMOUS,ANONYMOUS,ANONYMOUS@ANONYMOUS.COM,2023-10-23 18:44:55.294000+00:00


In [79]:
movies = pd.read_json(os.path.join(JSON_PATH, "movies.json"), orient="records", lines=True)
for col in ["runtime", "budget", "revenue", "vote_count"]:
    movies[col] = np.where(pd.isnull(movies[col]), -1, movies[col]).astype(int)
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6235 entries, 0 to 6234
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   tmdb_id       6235 non-null   int64              
 1   title         6235 non-null   object             
 2   release_date  6235 non-null   object             
 3   runtime       6235 non-null   int64              
 4   genres        6235 non-null   object             
 5   keywords      5999 non-null   object             
 6   overview      6235 non-null   object             
 7   budget        6235 non-null   int64              
 8   revenue       6235 non-null   int64              
 9   popularity    6235 non-null   float64            
 10  vote_average  6231 non-null   float64            
 11  vote_count    6235 non-null   int64              
 12  updated_at    6235 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(2), int64(5), object(5)
memo

In [80]:
ratings = pd.read_json(os.path.join(JSON_PATH, "ratings.json"), orient="records", lines=True)
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15372666 entries, 0 to 15372665
Data columns (total 4 columns):
 #   Column      Dtype              
---  ------      -----              
 0   user_id     int64              
 1   tmdb_id     int64              
 2   rating      float64            
 3   updated_at  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), int64(2)
memory usage: 469.1 MB


#### Create a SQLAlchemy Database Engine

In [82]:
from sqlalchemy import Engine, create_engine
from google.cloud.sql.connector import Connector
from pg8000 import Connection

In [83]:
def make_connection() -> Connection:
    """generate a new pg8000 connection for a CloudSQL instance"""

    project = "robot-ebert"
    region = "us-west1"
    instance = "robot-ebert"
    instance_connection_string = f"{project}:{region}:{instance}"

    connector = Connector()
    cnx = connector.connect(
        instance_connection_string=instance_connection_string,
        driver="pg8000",
        user="postgres",
        password=os.environ["POSTGRES_PASSWORD"],
        db="app"
    )
    return cnx


def get_engine(test: bool = False, echo: bool = False) -> Engine:
    """get a new SQLAlchemy Engine to manage DB connections"""

    if test:
        engine = create_engine(f"duckdb:///../app/database.duckdb", echo=echo)
    else:
        engine = create_engine("postgresql+pg8000://", creator=make_connection, echo=echo)
    return engine


In [84]:
engine = get_engine(test=False, echo=False)
engine

Engine(postgresql+pg8000://)

#### Insert Static Data into the Application Database

In [81]:
# users.to_sql(name="users", con=engine, index=False, if_exists="append", chunksize=1000, method="multi")

In [62]:
# movies.to_sql(name="movies", con=engine, index=False, if_exists="append", chunksize=1000, method="multi")

In [85]:
# ratings.to_sql(name="ratings", con=engine, index=False, if_exists="append", chunksize=1000, method="multi")

15372666

#### Insert Static Data Using SQLAlchemy

In [63]:
sys.path.append("..")

In [64]:
from sqlalchemy import insert, select, update, delete
from app import database

In [69]:
# with engine.begin() as cnx:
#     cnx.execute(database.movies.insert(), movies.to_dict(orient="records"))

### Test Out Query Functionality Using SQLAlchemy

In [71]:
from sqlalchemy import text

In [93]:
%time

with engine.connect() as cnx:
    result = cnx.execute(text("SELECT * FROM ratings WHERE user_id = '1'"))
    for row in result:
        print(row)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 13.8 µs
('1', '272', 4.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '9685', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '2108', 4.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '64678', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '84892', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '12444', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '13475', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '141', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '1726', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '9377', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '238', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '240', 5.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))
('1', '68718', 4.0, datetime.datetime(2023, 10, 23, 18, 44, 55, 294000))